In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
pd.set_option('display.max_columns', None)

In [3]:
disasters = gpd.read_file("disasters.geojson")

In [7]:
disastersChina = disasters.query("country == 'China'").copy(deep=True)

In [31]:
provinceData = pd.pivot_table(disastersChina, index = 'adm1', values='disasterno', columns='disastertype', aggfunc='count').reset_index()
provinceData = provinceData.rename_axis(None, axis=1)
provinceData.fillna(0, inplace=True)

In [32]:
provinceData

,adm1,drought,earthquake,extreme temperature,flood,landslide,mass movement (dry),storm
0,Anhui,7.0,1.0,2.0,33.0,0.0,0.0,27.0
1,Beijing,0.0,3.0,3.0,5.0,0.0,0.0,6.0
2,Chongqing,2.0,2.0,1.0,37.0,6.0,0.0,14.0
3,Fujian,3.0,1.0,0.0,44.0,4.0,0.0,66.0
4,Gansu,4.0,11.0,0.0,31.0,2.0,1.0,7.0
5,Guangdong,12.0,4.0,1.0,58.0,2.0,0.0,88.0
6,Guangxi,2.0,1.0,2.0,51.0,3.0,0.0,34.0
7,Guizhou,3.0,5.0,2.0,71.0,10.0,1.0,20.0
8,Hainan,0.0,0.0,1.0,6.0,0.0,0.0,30.0
9,Hebei,5.0,7.0,1.0,12.0,0.0,0.0,14.0


In [33]:
province = gpd.read_file("map_CHN_1.json")
province = province[['NAME_1','geometry']]
province.rename(columns={'NAME_1':'adm1'}, inplace=True)
provinceDisaster = province.merge(provinceData, on='adm1')

In [53]:
provinceDisaster['total'] = provinceDisaster['drought'] + provinceDisaster['earthquake']+ provinceDisaster['extreme temperature ']+ provinceDisaster['flood']+ provinceDisaster['storm']+ provinceDisaster['landslide']+ provinceDisaster['mass movement (dry)']
provinceDisaster['dominant_type'] = provinceDisaster[['drought', 'earthquake', 'extreme temperature ',
       'flood', 'landslide', 'mass movement (dry)', 'storm']].idxmax(axis=1)
provinceDisaster['point'] = provinceDisaster.representative_point()
provinceDisaster['geometry'] = provinceDisaster['point']
provinceDisaster.drop(columns=['point'], inplace=True)

In [54]:
provinceDisaster.to_file('provinceDisaster.geojson', driver='GeoJSON')
disastersChina.to_file('placeDisaster.geojson', driver='GeoJSON')